![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.Clinical_Named_Entity_Recognition_Model.ipynb)

## **Classify Public Health Mentions in German**

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs 

In [2]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving 4.2.0.spark_nlp_for_healthcare.json to 4.2.0.spark_nlp_for_healthcare.json


In [3]:
from johnsnowlabs import * 

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
jsl.install()

👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up if John Snow Labs home exists in /root/.johnsnowlabs this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-4.2.0-py3-none-any.whl to /usr/bin/python3
Running: /usr/bin/python3 -m pip install /root/.johnsnowlabs/py_installs/spark_nlp_jsl-4.2.0-py3-none-any.whl
👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare.json
Installing /root/.johnsnowlabs/py_installs/spark_ocr-4.1.0-py3-none-any.whl to /usr/bin/python3
Running: /usr/bin/python3 -m pip install /root/.johnsnowlabs/py_installs/spark_ocr-4.1.0-py3-none-any.whl
👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare.json
Installing /root/.johnsnowlabs/py_installs/spark_nlp-4.2.0

In [1]:
from johnsnowlabs import * 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to
spark = jsl.start(exclude_ocr = True)

👌 Detected license file /content/4.2.0.spark_nlp_for_healthcare.json
👌 Launched cpu-Optimized JVM SparkSession with Jars for: 🚀Spark-NLP==4.2.0, 💊Spark-Healthcare==4.2.0, running on ⚡ PySpark==3.1.2


In [2]:
text_list = ["""Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur der Schrecke nach dem Stich.""",
             """Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.""",
             """Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik würde so nicht gelöst.""",
             """Wir waren bei einem Wine Tasting in der Steiermark.""",
             """Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerprobleme habe.""",  
             """Es dürfte dafür aber keine relativ einfache Lösung geben - man muss wohl nur die Lüftung austauschen."""]

## **bert_sequence_classifier_health_mentions_bert**

In [12]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

sequenceClassifier = medical.BertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_bert", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])


bert_sequence_classifier_health_mentions_bert download started this may take some time.
[OK!]


In [4]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [5]:
result = pipeline.fit(df).transform(df)

In [6]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E

## **bert_sequence_classifier_health_mentions_gbert**

In [7]:
sequenceClassifier = medical.BertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_gbert", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])

bert_sequence_classifier_health_mentions_gbert download started this may take some time.
[OK!]


In [8]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [9]:
result = pipeline.fit(df).transform(df)

In [10]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E

## **bert_sequence_classifier_health_mentions_gbert_large**

In [11]:
sequenceClassifier = medical.BertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_gbert_large", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])

bert_sequence_classifier_health_mentions_gbert_large download started this may take some time.
[OK!]


In [12]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [13]:
result = pipeline.fit(df).transform(df)

In [14]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E

## **bert_sequence_classifier_health_mentions_medbert**

In [13]:
sequenceClassifier = medical.BertForSequenceClassification.pretrained("bert_sequence_classifier_health_mentions_medbert", "de", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class_")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    sequenceClassifier    
])

bert_sequence_classifier_health_mentions_medbert download started this may take some time.
[OK!]


In [14]:
files = [i for i in (range(1, len(text_list)+1))]

df = spark.createDataFrame(pd.DataFrame({'index' : files,'text': text_list}))

In [15]:
result = pipeline.fit(df).transform(df)

In [16]:
result.select('index',F.explode(F.arrays_zip(result.document.result, 
                                             result.class_.result)).alias("col"))\
      .select('index',F.expr("col['0']").alias("sentence"),
                      F.expr("col['1']").alias("prediction")).show(truncate=100)

+-----+----------------------------------------------------------------------------------------------------+--------------+
|index|                                                                                            sentence|    prediction|
+-----+----------------------------------------------------------------------------------------------------+--------------+
|    1|Die Schmerzen im Oberschenkel sind dann aber shcnell besser geworden, vielleicht war es auhc nur ...|health-related|
|    2| Der roman handelt von einer mutter die ihre tochter sucht nachdem diese im urlaub verschwunden ist.|    non-health|
|    3|Der Orthopäde/ Chirurg bei dem ich zuerst war, wollte nicht operieren- er meinte die Probelmatik ...|health-related|
|    4|                                                 Wir waren bei einem Wine Tasting in der Steiermark.|    non-health|
|    5|Auch der Betriebsarzt ist zu der Diagnose gekommen und sich weitgehend sicher, dass ich Zuckerpro...|health-related|
|    6|E